In [ ]:
# Import Libraries
from PIL import Image, ImageDraw
from pynq import Overlay
from pynq.drivers import HDMI
from pynq.drivers.video import VMODE_640x480
from pynq.drivers import Frame

import random
import cv2
import sys
import io

%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np

from pynq.board import LED
from pynq.board import RGBLED
from pynq.board import Switch
from pynq.board import Button
import time

Overlay("base.bit").download()

MAX_LEDS = 4
MAX_SWITCHES = 2
MAX_BUTTONS = 4

leds = [0] * MAX_LEDS
switches = [0] * MAX_SWITCHES
buttons = [0] * MAX_BUTTONS

for i in range(MAX_LEDS):
    leds[i] = LED(i)              
for i in range(MAX_SWITCHES):
    switches[i] = Switch(i)      
for i in range(MAX_BUTTONS):
    buttons[i] = Button(i)
    
# monitor (output) frame buffer size
frame_out_w = 1920
frame_out_h = 1080
    
# Helper function to clear LEDs
def clear_LEDs(LED_nos=list(range(MAX_LEDS))):
    """Clear LEDS LD3-0 or the LEDs whose numbers appear in the list"""
    for i in LED_nos:
        leds[i].off()
        
clear_LEDs()
    
# Helper functions    
def take_picture(pic_num):
    if pic_num == 0:
        # Take webcam picture and display on HDMI out
        videoIn = cv2.VideoCapture(0)
        ret, frame_vga = videoIn.read()

        frame_1920p = np.zeros((1080,1920,3)).astype(np.uint8)       
        frame_1920p[0:480,0:640,:] = frame_vga[0:480,0:640,:]
        hdmi_out.frame_raw(bytearray(frame_1920p.astype(np.int8).tobytes()))
        frame = hdmi_out.frame()
        
        orig_img_path = '/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num)+'snow.jpg'
        frame.save_as_jpeg(orig_img_path)
        videoIn.release()
    
    else:
        # Take webcam picture
        orig_img_path = '/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num)+'snow.jpg'
        !fswebcam  --no-banner --save {orig_img_path} -d /dev/video0 2> /dev/null

        img = Image.open('/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num)+'snow.jpg')
        img = img.resize((640,480))
        img.save('/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num)+'snow.jpg',"JPEG")
    
def blackwhite(pic_num):
    # Convert image to black and white only
    image = Image.open('/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num)+'snow.jpg')
    
    for v in range (width):
        for w in range (height):
            r = image.getpixel((v,w))[0]
            g = image.getpixel((v,w))[1]
            b = image.getpixel((v,w))[2]
            
            if r<128 and g<128 and b<128:
                r = 0
                g = 0
                b = 0
            else:
                r = 255
                g = 255
                b = 255
            image.putpixel((v,w),(r,g,b))
            
    orig_img_path = '/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num)+'snow.jpg'
    image.save(orig_img_path)

In [ ]:
while True:
    if switches[0].read() and switches[1].read():
        print("Ready.")
        
        # Initialize
        width = 640
        height = 480
        num = 100
        pic_num = 0
        flag = 0

        hdmi_out = HDMI('out')
        hdmi_out.mode(VMODE_640x480)
        hdmi_out.start()

        # Create an empty array
        snow_list = []

        # Loop and add a snow flake in a random x,y position
        for i in range(num):
            x = random.randrange(0, width)
            y = random.randrange(-height-10, -10)
            snow_list.append([x, y])
            
        while True:
            # Take picture
            if buttons[0].read() and flag == 0:
                leds[0].on()
                take_picture(pic_num)
                print("Picture taken.")
                flag = 1
                clear_LEDs()

            # Convert to black and white
            if buttons[1].read() and flag == 1:
                leds[1].on()
                blackwhite(pic_num)
                print("Picture converted to black and white.")
                pic_num = pic_num + 1
                flag = 2
                clear_LEDs()

            # Add effect
            if buttons[2].read() and flag == 2:
                leds[2].on()
                flag = 3
                for a in range (0,150):
                    image = Image.open('/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num-1)+'snow.jpg')

                    if buttons[3].read() and flag == 3:
                        leds[3].on()
                        take_picture(pic_num)
                        blackwhite(pic_num)
                        print("Another picture was taken and converted.")
                        pic_num = pic_num + 1
                        image = Image.open('/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num-1)+'snow.jpg')
                        leds[3].off()

                    for i in range(len(snow_list)):
                        # Draw the snow flake
                        draw = ImageDraw.Draw(image)
                        draw.ellipse((snow_list[i][0]-3, snow_list[i][1]-3, snow_list[i][0]+3, snow_list[i][1]+3), fill = 'blue', outline ='blue')

                        # Move the snow flake down one pixel
                        snow_list[i][1] += 5

                        # Check collision
                        for u in range(-3,4):
                            xcord = snow_list[i][0]
                            ycord = snow_list[i][1]+u

                            if ycord<height and ycord>0:
                                r = image.getpixel((xcord,ycord))[0]
                                g = image.getpixel((xcord,ycord))[1]
                                b = image.getpixel((xcord,ycord))[2]

                                if r==0 and g==0 and b==0:
                                    snow_list[i][0]=random.randrange(0, width)
                                    snow_list[i][1]=random.randrange(-50, -10)

                        # If the snow flake has moved off the bottom of the screen
                        if snow_list[i][1] > height:
                            # Reset it just above the top
                            y = random.randrange(-50, -10)
                            snow_list[i][1] = y
                            # Give it a new x position
                            x = random.randrange(0, width)
                            snow_list[i][0] = x

                    orig_img_path = '/home/xilinx/jupyter_notebooks/Pictures/'+str(pic_num-1)+'new_snow.jpg'
                    image.save(orig_img_path)

                    cap = cv2.VideoCapture(orig_img_path)

                    ret, frame_img = cap.read()
                    frame_1920p = np.zeros((1080,1920,3)).astype(np.uint8)       
                    frame_1920p[0:480,0:640,:] = frame_img[0:480,0:640,:]
                    hdmi_out.frame_raw(bytearray(frame_1920p.astype(np.int8).tobytes()))

                flag = 0
                print("Done.")
                clear_LEDs()

            clear_LEDs()

            # Break/release webcam and HDMI out
            if not switches[0].read() and flag == 0:
                cap.release()
                hdmi_out.stop()
                del hdmi_out
                leds[0].on()
                leds[1].on()
                leds[2].on()
                leds[3].on()
                break
                
    if not switches[0].read() and not switches[1].read():
        break